## Code for lstm network development. data preparation for lstm can be found in preprocess/seqdata.ipynb

In [3]:
import numpy as np
import pandas as pd
import time
import tensorflow as tf
from tensorflow import keras
from math import ceil, sqrt

import random
import glob,os
import matplotlib.pyplot as plt
import pickle
import pdb
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding, Masking, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
#from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
#from sklearn.externals import joblib
import joblib
import seaborn as sns
import matplotlib.animation as animation

import talos as ta
from talos.utils.gpu_utils import multi_gpu

Using TensorFlow backend.


### Load Data

In [4]:
# with open('/home/arash/ProjectVR/cleaneddata/seperateddata/inputseqtrain', 'rb') as f:
#     inputseqtrain = pickle.load(f) 

# with open('/home/arash/ProjectVR/cleaneddata/seperateddata/ytrain', 'rb') as f:
#     ytrain = pickle.load(f) 
# with open('/home/arash/ProjectVR/cleaneddata/seperateddata/inputseqTEST', 'rb') as f:
#     inputseqTEST = pickle.load(f) 
# with open('/home/arash/ProjectVR/cleaneddata/seperateddata/ytest', 'rb') as f:
#     ytest = pickle.load(f) 

folder = '/home/arash/ProjectVR/cleaneddata/seperateddata/'
dtype = '_LW_3'   
# TW_i_o for time seq, i sec input, o sec output length 
# LW_n for length seq, n/10 input proportion

with open(f'{folder}inputseqTrain{dtype}', 'rb') as f:
    inputseqTrain = pickle.load(f) 
    
with open(f'{folder}inputauxTrain{dtype}', 'rb') as f:
    inputauxTrain = pickle.load(f) 
    
with open(f'{folder}outTrain{dtype}', 'rb') as f:
    outTrain = pickle.load(f) 
    
with open(f'{folder}inputseqTest{dtype}', 'rb') as f:
    inputseqTest = pickle.load(f) 

with open(f'{folder}inputauxTest{dtype}', 'rb') as f:
    inputauxTest = pickle.load(f)
    
with open(f'{folder}outTest{dtype}', 'rb') as f:
    outTest = pickle.load(f)

In [5]:
# Merge aux and seq in to use as input
maxlen = max(inputseqTrain.shape[1],inputauxTrain.shape[1])

inputseqTrain = pad_sequences(inputseqTrain, maxlen = maxlen,
                              dtype='float32',value=-0.01)

inputauxTrain = pad_sequences(inputauxTrain, maxlen = maxlen,
                              dtype='float32',value=-0.01)

inpTrain = np.dstack((inputseqTrain,inputauxTrain))

inputseqTest = pad_sequences(inputseqTest, maxlen = maxlen,
                              dtype='float32',value=-0.01)

inputauxTest = pad_sequences(inputauxTest, maxlen = maxlen,
                              dtype='float32',value=-0.01)

inpTest = np.dstack((inputseqTest,inputauxTest))

### Define Vanilla Model

In [6]:
p = {"Llayers" : [1,2],
      "nodes" : [10,50],
      "batch_size": [128],
      "epochs": [200]}

def VanillaLSTM(X_t,Out_t,X_val,Out_val,params):
    n_features = X_t.shape[2] 
    n_steps_in = X_t.shape[1]
    n_steps_out = Out_t.shape[1]
    
    Llayers = params['Llayers']
    nodes = params['nodes']
    epochs = params['epochs']
    batch_size = params['batch_size']

    
    model = Sequential()
    model.add(Masking(mask_value=-0.01, input_shape=(n_steps_in, n_features)))
    
    for i in range(Llayers-1):
        model.add(LSTM(nodes, activation='relu',
                       return_sequences=True))
        
    model.add(LSTM(nodes, activation='relu'))
    model.add(Dense(n_steps_out))


    model.compile(optimizer='adam', loss='mse')
    
    Out_t = Out_t[:,:,0] # 0 is index for X
    Out_val = Out_val[:,:,0]
    
    start = time.time()
    history = model.fit(X_t, Out_t,
                        validation_data=[X_val, Out_val],
                        epochs = epochs, verbose = 100,
                        batch_size = batch_size)
    end = time.time()

    print ("Finished Fitting Model. # of Epochs: %d\n Time Taken : %d secs"
           % (epochs,end - start))
    
    return history, model

In [7]:
scan_object = ta.Scan(x = inputseqTrain,
                      y = outTrain, 
                      params = p,
                      model = VanillaLSTM,
                      val_split = 0.2,
                      experiment_name='VLSTM_0')

  0%|          | 0/4 [00:00<?, ?it/s]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2087 samples, validate on 522 samples
Epoch 1/200
Epoch 2/200
Epoch 3/200
Epoch 4/200
Epoch 5/200
Epoch 6/200
Epoch 7/200
Epoch 8/200
Epoch 9/200
Epoch 10/200
Epoch 11/200
Epoch 12/200
Epoch 13/200
Epoch 14/200
Epoch 15/200
Epoch 16/200
Epoch 17/200
Epoch 18/200
Epoch 19/200
Epoch 20/200
Epoch 21/200
Epoch 22/200
Epoch 23/200
Epoch 24/200
Epoch 25/200
Epoch 26/200
Epoch 27/200
Epoch 28/200
Epoch 29/200
Epoch 30/200
Epoch 31/200
Epoch 32/200
Epoch 33/200
Epoch 34/200
Epoch 35/200
Epoch 36/200
Epoch 37/200
Epoch 38/200
Epoch 39/200
Epoch 40/200
Epoch 41/200
Epoch 42/200
Epoch 43/200
Epoch 44/200
Epoch 45/200
Epoch 46/200
Epoch 47/200
Epoch 48/200
Epoch 49/200
Epoch 50/200
Epoch 51/200
Epoch 52/200
Epoch 53/200
Epoch 54/200
Epoch 55/200
Epoch 56/200
Epoch 57/200
Epoch 58/

 25%|██▌       | 1/4 [02:25<07:17, 145.84s/it]

Finished Fitting Model. # of Epochs: 200
 Time Taken : 145 secs
Train on 2087 samples, validate on 522 samples
Epoch 1/200
Epoch 2/200
Epoch 3/200
Epoch 4/200
Epoch 5/200
Epoch 6/200
Epoch 7/200
Epoch 8/200
Epoch 9/200
Epoch 10/200
Epoch 11/200
Epoch 12/200
Epoch 13/200
Epoch 14/200
Epoch 15/200
Epoch 16/200
Epoch 17/200
Epoch 18/200
Epoch 19/200
Epoch 20/200
Epoch 21/200
Epoch 22/200
Epoch 23/200
Epoch 24/200
Epoch 25/200
Epoch 26/200
Epoch 27/200
Epoch 28/200
Epoch 29/200
Epoch 30/200
Epoch 31/200
Epoch 32/200
Epoch 33/200
Epoch 34/200
Epoch 35/200
Epoch 36/200
Epoch 37/200
Epoch 38/200
Epoch 39/200
Epoch 40/200
Epoch 41/200
Epoch 42/200
Epoch 43/200
Epoch 44/200
Epoch 45/200
Epoch 46/200
Epoch 47/200
Epoch 48/200
Epoch 49/200
Epoch 50/200
Epoch 51/200
Epoch 52/200
Epoch 53/200
Epoch 54/200
Epoch 55/200
Epoch 56/200
Epoch 57/200
Epoch 58/200
Epoch 59/200
Epoch 60/200
Epoch 61/200
Epoch 62/200
Epoch 63/200
Epoch 64/200
Epoch 65/200
Epoch 66/200
Epoch 67/200
Epoch 68/200
Epoch 69/200
E

 50%|█████     | 2/4 [05:36<05:18, 159.29s/it]

Finished Fitting Model. # of Epochs: 200
 Time Taken : 190 secs
Train on 2087 samples, validate on 522 samples
Epoch 1/200
Epoch 2/200
Epoch 3/200
Epoch 4/200
Epoch 5/200
Epoch 6/200
Epoch 7/200
Epoch 8/200
Epoch 9/200
Epoch 10/200
Epoch 11/200
Epoch 12/200
Epoch 13/200
Epoch 14/200
Epoch 15/200
Epoch 16/200
Epoch 17/200
Epoch 18/200
Epoch 19/200
Epoch 20/200
Epoch 21/200
Epoch 22/200
Epoch 23/200
Epoch 24/200
Epoch 25/200
Epoch 26/200
Epoch 27/200
Epoch 28/200
Epoch 29/200
Epoch 30/200
Epoch 31/200
Epoch 32/200
Epoch 33/200
Epoch 34/200
Epoch 35/200
Epoch 36/200
Epoch 37/200
Epoch 38/200
Epoch 39/200
Epoch 40/200
Epoch 41/200
Epoch 42/200
Epoch 43/200
Epoch 44/200
Epoch 45/200
Epoch 46/200
Epoch 47/200
Epoch 48/200
Epoch 49/200
Epoch 50/200
Epoch 51/200
Epoch 52/200
Epoch 53/200
Epoch 54/200
Epoch 55/200
Epoch 56/200
Epoch 57/200
Epoch 58/200
Epoch 59/200
Epoch 60/200
Epoch 61/200
Epoch 62/200
Epoch 63/200
Epoch 64/200
Epoch 65/200
Epoch 66/200
Epoch 67/200
Epoch 68/200
Epoch 69/200
E

 75%|███████▌  | 3/4 [10:21<03:17, 197.05s/it]

Finished Fitting Model. # of Epochs: 200
 Time Taken : 284 secs
Train on 2087 samples, validate on 522 samples
Epoch 1/200
Epoch 2/200
Epoch 3/200
Epoch 4/200
Epoch 5/200
Epoch 6/200
Epoch 7/200
Epoch 8/200
Epoch 9/200
Epoch 10/200
Epoch 11/200
Epoch 12/200
Epoch 13/200
Epoch 14/200
Epoch 15/200
Epoch 16/200
Epoch 17/200
Epoch 18/200
Epoch 19/200
Epoch 20/200
Epoch 21/200
Epoch 22/200
Epoch 23/200
Epoch 24/200
Epoch 25/200
Epoch 26/200
Epoch 27/200
Epoch 28/200
Epoch 29/200
Epoch 30/200
Epoch 31/200
Epoch 32/200
Epoch 33/200
Epoch 34/200
Epoch 35/200
Epoch 36/200
Epoch 37/200
Epoch 38/200
Epoch 39/200
Epoch 40/200
Epoch 41/200
Epoch 42/200
Epoch 43/200
Epoch 44/200
Epoch 45/200
Epoch 46/200
Epoch 47/200
Epoch 48/200
Epoch 49/200
Epoch 50/200
Epoch 51/200
Epoch 52/200
Epoch 53/200
Epoch 54/200
Epoch 55/200
Epoch 56/200
Epoch 57/200
Epoch 58/200
Epoch 59/200
Epoch 60/200
Epoch 61/200
Epoch 62/200
Epoch 63/200
Epoch 64/200
Epoch 65/200
Epoch 66/200
Epoch 67/200
Epoch 68/200
Epoch 69/200
E

100%|██████████| 4/4 [16:52<00:00, 253.20s/it]

Finished Fitting Model. # of Epochs: 200
 Time Taken : 390 secs


In [10]:
# accessing the results data frame
scan_object.data.head()

,round_epochs,loss,val_loss,Llayers,batch_size,epochs,nodes
0,200,0.001047,0.001237,1,128,200,10
1,200,0.001443,0.001423,1,128,200,50
2,200,0.001203,0.001215,2,128,200,10
3,200,0.001155,0.001252,2,128,200,50


In [11]:
# accessing epoch entropy values for each round
scan_object.learning_entropy

,loss
0,0.004635
1,16.458583
2,9.767579
3,0.418930


In [12]:
# access the summary details
scan_object.details

experiment_name                 VLSTM_0
random_method          uniform_mersenne
reduction_method                   None
reduction_interval                   50
reduction_window                     20
reduction_threshold                 0.2
reduction_metric                val_acc
complete_time            07/10/20/19:44
x_shape                  (2609, 126, 6)
y_shape                  (2609, 114, 6)
dtype: object

In [ ]:
# class VanillaLSTM(BaseEstimator):
    
#     def __init__(self, nodes=50, batch_size=128,
#                  epochs=200, Llayers=1, features=6, steps_in=0, steps_out=0
#                  ):

#         self.nodes = nodes
#         self.batch_size = batch_size
#         self.epochs = epochs
#         self.Llayers= Llayers
#         self.features = features
#         self.steps_in = steps_in
#         self.steps_out = steps_out
#         self.maxXY=654.9
#         self.minXY=650.1


#     def create_model(self):
#         model = Sequential()
#         model.add(Masking(mask_value=-0.01, input_shape=(self.steps_in, self.features)))
#         for i in range(self.Llayers-1):
#             model.add(LSTM(self.nodes, activation='relu', return_sequences=True))
#         model.add(LSTM(self.nodes, activation='relu'))
#         model.add(Dense(self.steps_out))
#         model.compile(optimizer='adam', loss='mse')
#         return model
    
#     def fit(self,inputseq,out):
#         start = time.time()
#         X = inputseq[:,:,:self.features]
#         out = out[:,:,0] # 0 is index for X
#         self.model=self.create_model()
#         self.model.fit(X, out, epochs = self.epochs, verbose = 100, batch_size = self.batch_size)
#         end = time.time()
#         print ("Finished Fitting Model. # of Epochs: %d\n Time Taken : %d secs"
#                % (self.epochs,end - start))
#         return self
    
#     def predict(self, inputseq):
#         X = inputseq[:,:,:self.features]
        
#         return self.model.predict(X)
    
#     def score(self,inputseq,out):
        
#         X = inputseq[:,:,:self.features]
#         outpred = self.predict(X)
#         out = out[:,:,0]
#         outinv = np.zeros(out.shape)
#         outpredinv = np.zeros(outpred.shape)
#         for i in range(len(out)):
#             pos = sum (n>=0 for n in out[i])     #position of the last actual value on y, not padded
#             outinv[i][:pos]= out[i][:pos] * (self.maxXY-self.minXY) + self.minXY
#             outpredinv[i][:pos] = outpred[i][:pos]* (self.maxXY-self.minXY) + self.minXY
            
#         rmseScore = -sqrt(mean_squared_error(outpredinv, outinv))    #negative of RMSE

#         return rmseScore
        

### fitting Vanilla LSTM, using grid search for hypperparameters

In [ ]:
# n_features = inpTrain.shape[2] - 1
# n_steps_in = inpTrain.shape[1]
# n_steps_out = outTrain.shape[1]

# VLmodel=VanillaLSTM(epochs=200, features=n_features,
#                     steps_in=n_steps_in, steps_out=n_steps_out)

In [ ]:
# tuned_params = {"Llayers" : [1,2],"nodes" : [10,50]}#,"batch_size": [128]}

#gs = GridSearchCV(VLmodel, tuned_params,cv = 8, refit= True, n_jobs=-1,verbose = 100)

In [ ]:
#np.load('/home/arash/ProjectVR/models/trainedmodels/VLSTM1.pkl',allow_pickle=True)      
# outTrain.shape

In [ ]:
# grid_result = gs.fit(inpTrain,outTrain)

In [ ]:
# np.save(f'/home/arash/ProjectVR/models/GridResults/VLSTM200_8{dtype}.npy',grid_result.cv_results_)      
#filename: ModelnameEpochs_CV_data type

In [ ]:
# bestVLSTM = grid_result.best_estimator_

In [ ]:
# joblib.dump(bestVLSTM, '/home/arash/ProjectVR/models/trainedmodels/VLSTM1.pkl')

In [ ]:
# bestVLSTM.score(inpTest,outTest)

# bestVLSTM.score(inpTest,outTest)

# VLmodel.fit(inpTrain,outTrain)

# VLmodel.score(inpTEST,outTest)

In [ ]:
# history = bestVLSTM.model.history.history
# # plot history
# plt.plot(history['loss'], label='train')
# #pyplot.plot(history.history['val_loss'], label='test')
# plt.legend()
# plt.show()


## lstm model with aux variables

In [74]:
sum(inpTrain[:,:,6][0]!=-0.01)

14

In [77]:
p = {"Llayers" : [1,2],
     "Dlayers" : [1],
     "nodes" : [10,50],
     "batch_size": [128], 
     "drpout" : [0, 0.2],
     "epochs": [200]}

def AuxLSTM(X_t,Out_t,X_val,Out_val,params):
    n_features = X_t.shape[2] - 1
    aux_size = sum(X_t[:,:,n_features][0]!=-0.01)
    n_steps_in = X_t.shape[1]
    n_steps_out = Out_t.shape[1]
    
    Llayers = params['Llayers']
    Dlayers = params['Dlayers']
    drpout = params['drpout']
    nodes = params['nodes']
    epochs = params['epochs']
    batch_size = params['batch_size']

    
    seq_input = Input(shape=(n_steps_in, n_features), dtype='float32', name='seq_input')
    mask = Masking(mask_value=-0.01)(seq_input)
    
    if Llayers==1:
        lstm_out = LSTM(nodes, activation='relu')(mask)
        
    else:
        lstm_out = LSTM(nodes,activation='relu',return_sequences=True)(mask)
        for i in range(Llayers-2):
            lstm_out = LSTM(nodes,activation='relu',return_sequences=True)(lstm_out)
        lstm_out = LSTM(nodes, activation='relu')(lstm_out)

    #output for lstm, corresponds to 0.2 of loss, used to smooth training and regularization:
    sec_output = Dense(n_steps_out, activation='sigmoid' ,name='sec_output')(lstm_out)   

    auxiliary_input = Input(shape=(aux_size,), name='aux_input')
    

    mrgLayer = keras.layers.concatenate([lstm_out, auxiliary_input])

    # We stack a deep densely-connected network on top
    for i in range(Dlayers):
        mrgLayer = Dense(nodes, activation='relu')(mrgLayer)
        if drpout>0:
            mrgLayer = Dropout(drpout)(mrgLayer)

    # And finally we add the main logistic regression layer
    main_output = Dense(n_steps_out, activation='sigmoid', name='main_output')(mrgLayer)

    modelaux = Model(inputs=[seq_input, auxiliary_input], outputs=[main_output, sec_output])


    modelaux.compile(optimizer='adam', loss='mse',
              loss_weights=[1., 0.2])
    
    
    aux_t = X_t[:,:,n_features][:,-aux_size:]
    aux_val = X_val[:,:,n_features][:,-aux_size:]
    
    X_t = X_t[:,:,:n_features]
    X_val = X_val[:,:,:n_features]
    
    
    Out_t = Out_t[:,:,0] # 0 is index for X
    Out_val = Out_val[:,:,0]
    
    start = time.time()
    history = modelaux.fit([X_t,aux_t], [Out_t,Out_t], 
                        validation_data = [[X_val, aux_val],[Out_val,Out_val]],
                        epochs = epochs,
                        verbose = 100, 
                        batch_size = batch_size)
    
    end = time.time()

    print ("Finished Fitting Model. # of Epochs: %d\n Time Taken : %d secs"
           % (epochs,end - start))
    
    return history, modelaux

In [78]:
scan_object = ta.Scan(x = inpTrain,
                      y = outTrain, 
                      params = p,
                      model = AuxLSTM,
                      val_split = 0.2,
                      experiment_name='AuxLSTM_0')










  0%|          | 0/8 [00:00<?, ?it/s]

Train on 2087 samples, validate on 522 samples
Epoch 1/200
Epoch 2/200
Epoch 3/200
Epoch 4/200
Epoch 5/200
Epoch 6/200
Epoch 7/200
Epoch 8/200
Epoch 9/200
Epoch 10/200
Epoch 11/200
Epoch 12/200
Epoch 13/200
Epoch 14/200
Epoch 15/200
Epoch 16/200
Epoch 17/200
Epoch 18/200
Epoch 19/200
Epoch 20/200
Epoch 21/200
Epoch 22/200
Epoch 23/200
Epoch 24/200
Epoch 25/200
Epoch 26/200
Epoch 27/200
Epoch 28/200
Epoch 29/200
Epoch 30/200
Epoch 31/200
Epoch 32/200
Epoch 33/200
Epoch 34/200
Epoch 35/200
Epoch 36/200
Epoch 37/200
Epoch 38/200
Epoch 39/200
Epoch 40/200
Epoch 41/200
Epoch 42/200
Epoch 43/200
Epoch 44/200
Epoch 45/200
Epoch 46/200
Epoch 47/200
Epoch 48/200
Epoch 49/200
Epoch 50/200
Epoch 51/200
Epoch 52/200
Epoch 53/200
Epoch 54/200
Epoch 55/200
Epoch 56/200
Epoch 57/200
Epoch 58/200
Epoch 59/200
Epoch 60/200
Epoch 61/200
Epoch 62/200
Epoch 63/200
Epoch 64/200
Epoch 65/200
Epoch 66/200
Epoch 67/200
Epoch 68/200
Epoch 69/200
Epoch 70/200
Epoch 71/200
Epoch 72/200
Epoch 73/200
Epoch 74/200











 12%|█▎        | 1/8 [02:26<17:04, 146.32s/it]

Finished Fitting Model. # of Epochs: 200
 Time Taken : 145 secs
Train on 2087 samples, validate on 522 samples
Epoch 1/200
Epoch 2/200
Epoch 3/200
Epoch 4/200
Epoch 5/200
Epoch 6/200
Epoch 7/200
Epoch 8/200
Epoch 9/200
Epoch 10/200
Epoch 11/200
Epoch 12/200
Epoch 13/200
Epoch 14/200
Epoch 15/200
Epoch 16/200
Epoch 17/200
Epoch 18/200
Epoch 19/200
Epoch 20/200
Epoch 21/200
Epoch 22/200
Epoch 23/200
Epoch 24/200
Epoch 25/200
Epoch 26/200
Epoch 27/200
Epoch 28/200
Epoch 29/200
Epoch 30/200
Epoch 31/200
Epoch 32/200
Epoch 33/200
Epoch 34/200
Epoch 35/200
Epoch 36/200
Epoch 37/200
Epoch 38/200
Epoch 39/200
Epoch 40/200
Epoch 41/200
Epoch 42/200
Epoch 43/200
Epoch 44/200
Epoch 45/200
Epoch 46/200
Epoch 47/200
Epoch 48/200
Epoch 49/200
Epoch 50/200
Epoch 51/200
Epoch 52/200
Epoch 53/200
Epoch 54/200
Epoch 55/200
Epoch 56/200
Epoch 57/200
Epoch 58/200
Epoch 59/200
Epoch 60/200
Epoch 61/200
Epoch 62/200
Epoch 63/200
Epoch 64/200
Epoch 65/200
Epoch 66/200
Epoch 67/200
Epoch 68/200
Epoch 69/200
E










 25%|██▌       | 2/8 [05:30<15:46, 157.71s/it]

...










 75%|███████▌  | 6/8 [21:48<08:17, 248.95s/it]

Train on 2087 samples, validate on 522 samples
Epoch 1/200
Epoch 2/200
Epoch 3/200
Epoch 4/200
Epoch 5/200
Epoch 6/200
Epoch 7/200
Epoch 8/200
Epoch 9/200
Epoch 10/200
Epoch 11/200
Epoch 12/200
Epoch 13/200
Epoch 14/200
Epoch 15/200
Epoch 16/200
Epoch 17/200
Epoch 18/200
Epoch 19/200
Epoch 20/200
Epoch 21/200
Epoch 22/200
Epoch 23/200
Epoch 24/200
Epoch 25/200
Epoch 26/200
Epoch 27/200
Epoch 28/200
Epoch 29/200
Epoch 30/200
Epoch 31/200
Epoch 32/200
Epoch 33/200
Epoch 34/200
Epoch 35/200
Epoch 36/200
Epoch 37/200
Epoch 38/200
Epoch 39/200
Epoch 40/200
Epoch 41/200
Epoch 42/200
Epoch 43/200
Epoch 44/200
Epoch 45/200
Epoch 46/200
Epoch 47/200
Epoch 48/200
Epoch 49/200
Epoch 50/200
Epoch 51/200
Epoch 52/200
Epoch 53/200
Epoch 54/200
Epoch 55/200
Epoch 56/200
Epoch 57/200
Epoch 58/200
Epoch 59/200
Epoch 60/200
Epoch 61/200
Epoch 62/200
Epoch 63/200
Epoch 64/200
Epoch 65/200
Epoch 66/200
Epoch 67/200
Epoch 68/200
Epoch 69/200
Epoch 70/200
Epoch 71/200
Epoch 72/200
Epoch 73/200
Epoch 74/200











 88%|████████▊ | 7/8 [26:28<04:18, 258.31s/it]

Train on 2087 samples, validate on 522 samples
Epoch 1/200
Epoch 2/200
Epoch 3/200
Epoch 4/200
Epoch 5/200
Epoch 6/200
Epoch 7/200
Epoch 8/200
Epoch 9/200
Epoch 10/200
Epoch 11/200
Epoch 12/200
Epoch 13/200
Epoch 14/200
Epoch 15/200
Epoch 16/200
Epoch 17/200
Epoch 18/200
Epoch 19/200
Epoch 20/200
Epoch 21/200
Epoch 22/200
Epoch 23/200
Epoch 24/200
Epoch 25/200
Epoch 26/200
Epoch 27/200
Epoch 28/200
Epoch 29/200
Epoch 30/200
Epoch 31/200
Epoch 32/200
Epoch 33/200
Epoch 34/200
Epoch 35/200
Epoch 36/200
Epoch 37/200
Epoch 38/200
Epoch 39/200
Epoch 40/200
Epoch 41/200
Epoch 42/200
Epoch 43/200
Epoch 44/200
Epoch 45/200
Epoch 46/200
Epoch 47/200
Epoch 48/200
Epoch 49/200
Epoch 50/200
Epoch 51/200
Epoch 52/200
Epoch 53/200
Epoch 54/200
Epoch 55/200
Epoch 56/200
Epoch 57/200
Epoch 58/200
Epoch 59/200
Epoch 60/200
Epoch 61/200
Epoch 62/200
Epoch 63/200
Epoch 64/200
Epoch 65/200
Epoch 66/200
Epoch 67/200
Epoch 68/200
Epoch 69/200
Epoch 70/200
Epoch 71/200
Epoch 72/200
Epoch 73/200
Epoch 74/200











100%|██████████| 8/8 [32:38<00:00, 244.78s/it]


In [79]:
# accessing the results data frame
scan_object.data.head()

,round_epochs,loss,main_output_loss,sec_output_loss,val_loss,val_main_output_loss,val_sec_output_loss,Dlayers,Llayers,batch_size,drpout,epochs,nodes
0,200,0.002019,0.001564,0.002102,0.001481,0.001158,0.001600,1,1,128,0.0,200,10
1,200,0.001566,0.001237,0.001476,0.001195,0.001111,0.001212,1,1,128,0.0,200,50
2,200,0.001904,0.001599,0.001525,0.001315,0.001270,0.001150,1,1,128,0.2,200,10
3,200,NaN,NaN,NaN,NaN,NaN,NaN,1,1,128,0.2,200,50
4,200,0.008250,0.007177,0.005327,0.007158,0.006451,0.004836,1,2,128,0.0,200,10


In [ ]:
# class AuxLSTM(BaseEstimator):
    
#     def __init__(self, nodes=50, batch_size=32,
#                  epochs=200, Dlayers=1, Llayers=1, features=6, steps_in=0, steps_out=0, drpout=0.5
#                  ):

#         self.nodes = nodes
#         self.features = features
#         self.steps_in = steps_in
#         self.steps_out = steps_out
#         self.epochs = epochs
#         self.batch_size = batch_size
#         self.Dlayers= Dlayers   #number of hidden dense layers
#         self.Llayers = Llayers  #number of lstm layers
#         self.maxXY=654.9
#         self.minXY=650.1
#         self.drpout=drpout


#     def create_model(self):
        
#         seq_input = Input(shape=(self.steps_in, self.features), dtype='float32', name='seq_input')
#         mask = Masking(mask_value=-0.01)(seq_input)
#         if self.Llayers==1:
            
#             lstm_out = LSTM(self.nodes, activation='relu')(mask)
            
#         else:
#             lstm_out = LSTM(self.nodes,activation='relu',return_sequences=True)(mask)
            
#             for i in range(self.Llayers-2):
#                 lstm_out = LSTM(self.nodes,activation='relu',return_sequences=True)(lstm_out)
                
#             lstm_out = LSTM(self.nodes, activation='relu')(lstm_out)

#         #output for lstm, corresponds to 0.2 of loss, used to smooth training and regularization:
#         sec_output = Dense(self.steps_out, activation='sigmoid' ,name='sec_output')(lstm_out)   

#         auxiliary_input = Input(shape=(14,), name='aux_input')
        
#         x = keras.layers.concatenate([lstm_out, auxiliary_input])

#         # We stack a deep densely-connected network on top
#         for i in range(self.Dlayers):
#             x = Dense(self.nodes, activation='relu')(x)
#             if self.drpout>0:
#                 x = Dropout(self.drpout)(x)

#         # And finally we add the main logistic regression layer
#         main_output = Dense(self.steps_out, activation='sigmoid', name='main_output')(x)

#         modelaux = Model(inputs=[seq_input, auxiliary_input], outputs=[main_output, sec_output])


#         modelaux.compile(optimizer='adam', loss='mse',
#                       loss_weights=[1., 0.2])
#         return modelaux
    
#     def fit(self,inp,out):
#         start = time.time()
#         X = inp[:,:,:self.features]
#         aux = inp[:,:,self.features][:,-14:] 
#         out = out[:,:,0]
#         self.model = self.create_model()
#         self.model.fit([X,aux], [out,out], epochs = self.epochs, verbose = 100, batch_size = self.batch_size)
#         end = time.time()
#         print ("Finished Fitting AuxModel. # of Epochs: %d\n Time Taken : %d secs"
#                % (self.epochs,end - start))
#         return self
    
#     def predict(self, inp):
#         X = inp[:,:,:self.features]
#         aux = inp[:,:,self.features][:,-14:]
        
#         return self.model.predict([X,aux])
    
#     def score(self,inp,out):
#         out = out[:,:,0]
#         outpred = self.predict(inp)[0]        #auxlstm has two identical outputs, 1st in chosen
#         outinv = np.zeros(out.shape)
#         outpredinv = np.zeros(outpred.shape)
#         for i in range(len(out)):
#             pos = sum (n>=0 for n in out[i])    #position of the last actual value on y, not padded
#             outinv[i][:pos]= out[i][:pos] * (self.maxXY-self.minXY) + self.minXY
#             outpredinv[i][:pos] = outpred[i][:pos]* (self.maxXY-self.minXY) + self.minXY
            
#         rmseScore = -sqrt(mean_squared_error(outpredinv, outinv))    #negative of RMSE
#         return rmseScore
        

In [ ]:
# n_features = (inpTrain.shape[2]-1)
# n_steps_in = inpTrain.shape[1]
# n_steps_out = outTrain.shape[1]

# Auxmodel=AuxLSTM(epochs=100, features=n_features,
#                     steps_in=n_steps_in, steps_out=n_steps_out)
 

In [ ]:
# tuned_params = {"Dlayers" : [1,2]}#,'drpout': [0,0.5], "Llayers" : [2]}
#                # 'batch_size' : [64], 'nodes': [100]  }

# gsaux = GridSearchCV(Auxmodel, tuned_params,cv = 5, refit= True, n_jobs=-1)

# gridaux_result = gsaux.fit(inpTrain,outTrain)

# gridaux_result.cv_results_

# bestauxLSTM = gridaux_result.best_estimator_

# bestauxLSTM.score(inpTest,outTest)

# np.save('/home/arash/ProjectVR/models/GridResults/Aux0_5LSTM001.npy',gridaux_result.cv_results_)
# bestauxLSTM = gridaux_result.best_estimator_

# joblib.dump(bestauxLSTM, '/home/arash/ProjectVR/models/trainedmodels/Aux0_5LSTM6.pkl')
# bestauxLSTM.score(inputseqTEST,ytest)

In [ ]:
X = inputseqtrain[:,:,:5]
aux = inputseqtrain[:,:,:5][:,:14]    
ypred = Aux1Best.predict(inputseqtrain)[1]        #auxlstm has two identical outputs, 1st in chosen
yinv = np.zeros(ytrain.shape)
ypredinv = np.zeros(ypred.shape)
for i in range(len(ytrain)):
     pos = sum (n>=0 for n in ytrain[i])                #position of the last actual value on y, not padded
     yinv[i][:pos]= ytrain[i][:pos] * (654.7-650.4) + 650.4
     ypredinv[i][:pos] = ypred[i][:pos]* (654.7-650.4) + 650.4
rmseScore = -sqrt(mean_squared_error(ypredinv, yinv))    #negative of RMSE


In [ ]:
aux1=np.load('/home/arash/ProjectVR/models/GridResults/AuxLSTM00L.npy',allow_pickle=True)

In [ ]:
np.zeros((1,10))

In [ ]:
Aux1Best=AuxLSTM(epochs=1000, features=n_features,
                    steps_in=n_steps_in, steps_out=n_steps_out,Llayers=2,Dlayers=1,drpout=0.5,
                   batch_size=32, nodes=50)

In [ ]:
Aux2Best=AuxLSTM(epochs=100, features=n_features,
                    steps_in=n_steps_in, steps_out=n_steps_out,Llayers=2,Dlayers=1,drpout=0.5,
                   batch_size=32, nodes=50)

# Animation Creation

In [ ]:
bestaux=joblib.load('/home/arash/ProjectVR/models/trainedmodels/AuxLSTM2.pkl')

In [ ]:
bestvlstm=joblib.load('/home/arash/ProjectVR/models/trainedmodels/VLSTM1.pkl')

In [ ]:
bestaux.score(inputseqtrain,ytrain)

In [ ]:
bestaux0_5=joblib.load('/home/arash/ProjectVR/models/trainedmodels/Aux0_5LSTM6.pkl')

In [ ]:
maxXY=654.9
minXY=650.1

In [ ]:
num=1

In [ ]:
ypred = bestaux.predict(inputseqtrain)[0][num]        #auxlstm has two identical outputs, 1st in chosen
ypredvlstm=bestvlstm.predict(inputseqtrain)[num] 


pos = sum (n>=0 for n in ytrain[num])                #position of the last actual value on y, not padded

title= "X Coordinate"
y= ypred[:pos]* (maxXY-minXY) + minXY
x = np.arange(0,len(y)/10,0.1)
xv=x
yv=ypredvlstm[:pos]* (maxXY-minXY) + minXY

ytrue = ytrain[num][:pos] * (maxXY-minXY) + minXYnj
xtrue = x

In [ ]:
def augment(xold,yold,numsteps):
    xnew = []
    ynew = []
    for i in range(len(xold)-1):
        difX = xold[i+1]-xold[i]
        stepsX = difX/numsteps
        difY = yold[i+1]-yold[i]
        stepsY = difY/numsteps
        for s in range(numsteps):
            xnew = np.append(xnew,xold[i]+s*stepsX)
            ynew = np.append(ynew,yold[i]+s*stepsY)
    return xnew,ynew

In [ ]:
x,y=augment(x,y,10)
xv,yv=augment(xv,yv,10)
xtrue,ytrue=augment(xtrue,ytrue,10)


In [ ]:
true0=pd.DataFrame(ytrue,xtrue)
pred0=pd.DataFrame(y,x)
Vpred0=pd.DataFrame(yv,xv)
true0.columns = {title}

pred0.columns={title}
Vpred0.columns={title}

plt.rcParams['animation.ffmpeg_path'] = '/home/arash/anaconda3/envs/gpusupport/bin/ffmpeg'    

In [ ]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=20, metadata=dict(artist='Arash'), bitrate=1800)

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(10,6))
plt.xlim(0, len(x)/100)
plt.ylim(np.min(pred0)[0], np.max(pred0)[0])
plt.xlabel('Time (s)',fontsize=14)
plt.ylabel(title,fontsize=14)
plt.title('Pedestrian Position Over Time',fontsize=14)
from matplotlib.lines import Line2D

custom_lines = [Line2D([0], [0], color='b', lw=4),
                Line2D([0], [0], color='g', lw=4),
                Line2D([0], [0], color='r', lw=4)]
plt.legend(custom_lines, ['AuxLSTM', 'True Trajectory', 'Vanilla LSTM'])

In [ ]:
def animate(i):
    dataaux = pred0.iloc[:int(i+1)] #select data range
    datatrue = true0.iloc[:int(i+1)] #select data range
    dataV=Vpred0.iloc[:int(i+1)] #select data range
    p = sns.lineplot(x=dataaux.index, y=dataaux[title], data=dataaux,markers=True,color="b")
    p2 = sns.lineplot(x=datatrue.index, y=datatrue[title], data=datatrue, markers=True,color="g")
    p3 = sns.lineplot(x=dataV.index, y=dataV[title], data=dataV, markers=True,color="r")
    sns.set_style("white")
    p.tick_params(labelsize=10)
    p2.tick_params(labelsize=10)
    p3.tick_params(labelsize=10)
    plt.setp(p.lines,linewidth=2)
    plt.setp(p2.lines,linewidth=2)

In [ ]:
ani = animation.FuncAnimation(fig, animate,frames=400 ,repeat=True)

ani.save('Sample03.mp4', writer=writer)

In [ ]:

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())






In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))